# Introduction to Recurrent Neural Networks

The final neural network architecture we will cover is the recurrent neural network.

## What we will accomplish

In this notebook we will:
- Discuss the kinds of problems recurrent nets are designed for,
- Give an overview of basic RNN architectures,
- Demonstrate the weighted sum setup for such architectures and
- Build a RNN to predict IMDB review sentiment.

Let's go!

In [ ]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
from seaborn import set_style

## This sets the plot style
## to have a grid on a white background
set_style("whitegrid")

## Sequential data

Similar to how convolutional neural networks were designed to deal with grid-based data, recurrent neural networks (RNN) were built to deal with sequential data. Some examples of sequential data include:
- Time series,
- Natural language,
- Music
- and More.

In sequential data it is often the case that the past impacts the future. For example, in text data preceding words may have an impact on our understanding of the current word.

## Simple RNN architecture

RNNs actually have a wide array of architectures depending upon the specific problem you are trying to solve. To demonstrate the general idea we will consider this simple architecture:

<img src="rnn1.png" width="40%"></img>

Here we have a RNN built to take in sequential data $(X^{(i)}, y^{(i)})$ four steps at a time, with four corresponding hidden layers. A use case of such an architecture would be an autofill predictor that takes in four words and then predicts what the next word will be. In this example, the $X^{(i)}$ and $y^{(i)}$ would be one-hot encoded vectors of length $d$, where $d$ is the size of the <i>lexicon</i>.

Each "step" of the architecture's sequence actually represents a unique feed forward network like so:

<img src="rnn2.png" width="50%"></img>

Where hidden layer $t+1$ also takes inputs from hidden layer $t$. In this way the final hidden layer takes input from all the previous hidden layers in a manner similar to a moving average model from our `Time Series` section.

### Weighted sum set up

The values for each hidden layer's nodes in this network are given by:

$$
h_1 = \Phi \left(W_{xh} X^{(1)}\right) \ \text{ and } \ h_t = \Phi \left( W_{xh}X^{(t)} + W_{hh} h_{t-1} \right),
$$

where $W_{xh}$ and $W_{hh}$ have the same entries regardless of the hidden layer considered, and the predicted value for each node of the output layers is given by:

$$
\hat{y^{(t)}} = \sigma\left( W_{hy} h_t \right),
$$

where the entries of $W_{hy}$ are the same regardless of $t$ and the nonlinear activation functions, $\Phi$ and $\sigma$ could be different or the same.

The dimensions of each $W_*$ weight matrix depend upon the dimensions of the $X^{(t)}$ and $y^{(t)}$ vectors and  the size of the hidden layer.

## Example: IMDB sentiment analysis

As an illustrative example we will use `keras` to build a sentiment classifier using IMDB movie reviews. Let's first load this data set.

In [ ]:
## The data is stored in here
from keras.datasets import imdb

In [ ]:
## This will determine the number of vocab words in our
## dictionary
max_features = 10000

## num_words tells keras to return the reviews so they contain only
## the num_words most used words across all the reviews
(X_train, y_train), (X_test,y_test) = imdb.load_data(num_words=max_features)

## Note you may receive a warning, this is not your fault, and is due to how
## keras is loading the data

In [ ]:
## Let's look at the first training observation
print(X_train[0])
print(y_train[0])

The data is stored as a list of indices, each of which is representative of a word. Let's see what this particular review looks like, once we have translated it from indices to words. Do not focus on the following code for now, as it is not important for building the neural network.

In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key,value) in word_index.items()])

## The first training review, where words outside the top 1000 are replaced with
## ? marks
print(" ".join([reverse_word_index.get(i-3, '?') for i in X_train[0]]))
print()
print("sentiment value =", y_train[0])

The review above had a $y$ value of $1$, meaning that it has positive sentiment. A value of $0$ indicates a negative sentiment.

### Our network architecture

The RNN architecture we will use for this problem is pictured below:

<img src="rnn3.png" width="60%"></img>

where $s$ denotes the length of the sequence we will consider.

In this problem we have a single output at the end of the sequence because we are only predicting whether the entire sequence represents a positive or negative review. This example demonstrates how different sequential prediction tasks can correspond to slightly different network architectures.

Let's now turn our data into sequences and then build our RNN.

In [ ]:
## Import pad_sequences
from keras.utils import pad_sequences

In [ ]:
print("The shape of our Xs prior to becoming sequences")
print("X_train:", np.shape(X_train))
print("X_test:", np.shape(X_test))

In [ ]:
## In order to fit a neural net to these observations
## we need all of our reviews to be sequences of the same length
## We'll set a fixed length of 100 words. For those shorter than 100
## words we'll add padding. For those longer than 100 words, we'll cut off the
## excess
## keras does this for us :)
max_length = 100

## pad_sequences(data, maxlen)
X_train_seq = 
X_test_seq = 

In [ ]:
print("The shape of our Xs after becoming sequences")
print("X_train:", np.shape(X_train_seq))
print("X_test:", np.shape(X_test_seq))

In [ ]:
## Note the padding at the beginning 
X_train_seq[100,:]

In [ ]:
## Making our validation set
from sklearn.model_selection import train_test_split

X_train_train,X_val,y_train_train,y_val = train_test_split(X_train_seq, y_train,
                                                           test_size=.2,
                                                           shuffle=True,
                                                           stratify = y_train,
                                                           random_state=440)

### Making the Network

This network will introduce two new layer types `Embedding` and `SimpleRNN`. 

For now all you will need to know about the `Embedding` layer is that it takes in the sequences we just generated and turns them into a vector representation of the sequence that is more useful to the network. One way to think of this is the `Embedding` layer is similar to running the data through a PCA type step prior to fitting the network. The exact details are slightly beyond the scope of our boot camp and better suited for a natural language processing course, here are the docs on the `Embedding` layer, <a href="https://keras.io/api/layers/core_layers/embedding/">https://keras.io/api/layers/core_layers/embedding/</a>. <i>Note: the `Embedding` layer is specific to NLP tasks, and not used if you are building a RNN on time series data. This is a preprocessing step.</i>

The `SimpleRNN` layer is the akin to the RNN architecture we described above. Here are the docs <a href="https://keras.io/api/layers/recurrent_layers/simple_rnn/">https://keras.io/api/layers/recurrent_layers/simple_rnn/</a>.

In [ ]:
## Import all the keras stuff we'll need
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

###### 1. Make the model object

In [ ]:
model = models.Sequential()

##### 2. Add the layers we want

In [ ]:
## Embedding is added first
## the size of our dictionary is the first input, max_features
## followed by the desired embedding size, I chose 32, again for no particular reason
model.add()

## Then the SimpleRNN layer
## the first input should be the same size as the input layer, i.e. 32
## return_sequences determines if the network should return the hidden state value for 
## each hidden layer h1, h2, ..., hT
## Since we only want the last one, hT, we set it equal to False
## If we were building a network with multiple SimpleRNN layers,
## you'd set this to True
model.add()

In [ ]:
model.summary()

In [ ]:
## Finally we'll add a Dense Layer for classification
## purposes
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

##### 3. Compile the model

Again we need to compile the model

In [ ]:
## notice we use binary_crossentropy here
## this is because our sentiment problem is a binary
## classification problem
model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

##### 4. Fit the model

In [ ]:
epochs = 10

history = model.fit(X_train_train, y_train_train,
                    epochs = epochs,
                    batch_size=128,
                    validation_data=(X_val,y_val))

## Note training this model can take a while

In [ ]:
history_dict = history.history

In [ ]:
## Plotting the training and validation accuracy
plt.figure(figsize = (8,6))

plt.scatter(range(1,epochs+1), history_dict['accuracy'], label = "Training Accuracy")
plt.scatter(range(1,epochs+1), history_dict['val_accuracy'], label = "Validation Set Accuracy")

plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.legend(fontsize=12)

plt.show()

In [ ]:
## Plotting the training and validation loss
plt.figure(figsize = (8,6))

plt.scatter(range(1,epochs+1), history_dict['loss'], label = "Training Loss")
plt.scatter(range(1,epochs+1), history_dict['val_loss'], label = "Validation Set Loss")

plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Loss Function Value", fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.legend(fontsize=12)

plt.show()

## Next steps

RNNs can be used for a variety of sequential data problems that we do not have time to cover in depth.

Moreover, `SimpleRNN` is actually a seldom used RNN layer type because there are more complicated `keras` layers that tend to outperform `SimpleRNN`. 

For those interested in learning more about RNN here are two references I leaned on heavily in making this notebook:
- <a href="http://faculty.neu.edu.cn/yury/AAI/Textbook/Deep%20Learning%20with%20Python.pdf">Deep Learning with Python</a>, which is a practical guide on how to implement neural networks with `keras`. RNNs start in chapter 6.
- <a href="https://link.springer.com/book/10.1007/978-3-319-94463-0">Neural Networks and Deep Learning</a>, which is a textbook that builds up the theory of neural networks. Recurrent neural networks start in chapter 7.

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)